In [ ]:
#pip install pickle-mixin
#pip install fuzzywuzzy
#pip install pyarrow

In [1]:
import pandas as pd
import os
import pickle
import json
from fuzzywuzzy import fuzz

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Business: Contiene información del comercio, incluyendo localización, atributos y categorías.


In [11]:
# Ruta al archivo .pkl que deseas abrir
ruta_archivo_pkl = '../PROYECTO_FINAL_EQUIPO/Yelp/business.pkl'

# Abrir el archivo .pkl en modo lectura binaria
with open(ruta_archivo_pkl, 'rb') as archivo_pkl:
    # Cargar el objeto desde el archivo .pkl
    business_Yelp = pickle.load(archivo_pkl)


In [12]:
business_Yelp.shape

(150346, 28)

In [13]:
business_Yelp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null

In [14]:
#Eliminando Columnas Duplicadas
business_Yelp = business_Yelp.loc[:, ~business_Yelp.columns.duplicated()]


In [15]:
business_Yelp.sample(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
117129,OHs8Aeu5lPM-BBfUqyU7Dw,3 Chefs Restaurant,323 Old York Rd,Jenkintown,CA,19046,40.091773,-75.125167,2.0,7,0,"{'Ambience': '{'romantic': False, 'intimate': ...","Sandwiches, Diners, Restaurants",None
77724,gdH0yhCQaHjBQ6vd0rYiSA,Krispy Krunchy Chicken,1825-33 West Oregon Ave,Philadelphia,PA,19145,39.918163,-75.178955,3.0,5,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsDel...","Breakfast & Brunch, Chicken Wings, American (N...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."


### Eliminar columnas que no nos sirven

In [17]:
#Esta columna indica si un negocio esta abierto o cerrado en el momento, así que no nos sirve.
business_Yelp.drop(columns='is_open',inplace=True)

In [18]:
#Bajo el mismo criterio que en Google, no se sabe cuando se tomaron las lecturas de Reviews
#por lo que es conveniente imputar estos datos dependiendo el periodo que se trabaje.
business_Yelp.drop(columns='review_count',inplace=True)
business_Yelp.drop(columns='stars',inplace=True)


Dejar solo los negocios que sean del tipo Restaurantes.

In [19]:
#Generar una lista de categorías únicas:

# Rellena los valores nulos con una cadena vacía
business_Yelp['categories'].fillna('', inplace=True)

# Dividir la columna "categories" en listas de categorías
categories_lists = business_Yelp['categories'].str.split(', ')

# Unir todas las listas en una única lista
all_categories = [category for categories_list in categories_lists for category in categories_list]

# Convertir el conjunto de nuevo en una lista si es necesario
unique_categories = set(all_categories)
unique_categories_list = list(unique_categories)

In [20]:
#Con las categorías únicas y nuestra palabra clave, obtendremos similitud de categorías
#Para asegurar seleccionar todos los restaurantes
keyword = "restaurant"

# Función para calcular la similitud entre la palabra clave y cada categoría
def calcular_similitud(categoria, keyword):
    return fuzz.ratio(keyword, categoria)

# Calcular la similitud para cada categoría en la lista
similitudes = [(categoria, calcular_similitud(categoria, keyword)) for categoria in unique_categories_list]

# Filtrar categorías que tienen una similitud mayor que un umbral (por ejemplo, 70)
umbral_similitud = 63  # Puedes ajustar este valor según tus necesidades
categorias_similares = [categoria for categoria, similitud in similitudes if similitud >= umbral_similitud]

In [21]:
categorias_similares

['Pop-Up Restaurants', 'Restaurants']

In [22]:
df_business_Yelp = business_Yelp
# Primero, dividimos los valores de 'categories' en listas de strings
df_business_Yelp['categories'] = df_business_Yelp['categories'].str.split(',')

df_business_Yelp = df_business_Yelp.dropna(subset=['categories'])

#Filtrar por categoría "Restaurantes"
df_business_Yelp = df_business_Yelp[df_business_Yelp['categories'].apply(lambda categories: 'Restaurants' in categories or 'Pop-Up Restaurants' in categories)]

# Unir los valores de la lista en categorías en un solo string separado por comas
df_business_Yelp['categories'] = df_business_Yelp['categories'].apply(lambda x: ','.join(x))

# Recetar el Indice
df_business_Yelp.reset_index(inplace=True, drop=True)

In [23]:
df_business_Yelp.shape

(15305, 11)

In [24]:
df_business_Yelp.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,AB,89502,39.476117,-119.789339,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Restaurants, Italian","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [25]:
#Aunque no hay duplicados, debemos quedarnos con los restaurantes que tengamos reseñas.
df_business_Yelp['business_id'].duplicated().sum()

0

## DataSet Review

In [26]:
# Ruta de la carpeta de datos de Google Maps
yelp_folder = '/Users/alejandrobenjaminnunezguzman/Desktop/PROYECTO_FINAL_EQUIPO/Yelp'

# Ruta de la sub-carpeta "metadata-sitios"
yelp_subfolder = os.path.join(yelp_folder, 'Reviews')

In [27]:
reviews = []

# Recorrer todos los archivos
for archivo in os.listdir(yelp_subfolder):
        # Ruta del archivo JSON actual
        ruta_archivo = os.path.join(yelp_subfolder, archivo)

        # Leer el contenido del archivo
        with open(ruta_archivo) as f:
            contenido_json = f.readlines()

        # Cargar cada objeto JSON por separado
        for linea in contenido_json:
            try:
                data = json.loads(linea)
            except json.JSONDecodeError as e:
                print(f"Error al cargar el archivo JSON {ruta_archivo}: {e}")
                continue

            reviews.append(data)

# Convertir los resultados a un pandas DataFrame
df_reviews = pd.DataFrame(reviews)

In [37]:
#Reviews de todos los negocios
df_reviews.shape

(6990280, 9)

Seleccionamos únicamente reviews de negocios del tipo: Restaurantes

In [57]:
df_business_Yelp.sample(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,attributes,categories,hours
14623,4DPO4PnukaY3mmm4MwP1ZA,Wrapture,"City Centre Mall East, 803-10088 102 Ave NW, #29",Edmonton,FL,T5J 2Z1,53.540895,-113.495596,"{'RestaurantsAttire': 'u'casual'', 'Alcohol': ...","Restaurants, Sandwiches, Delis",None


In [58]:
# Crear una lista de IDs a partir de df1
lista_de_ids = df_business_Yelp['business_id'].tolist()

# Filtrar df2 basándose en la lista de IDs de df1
df_reviews_res = df_reviews[df_reviews['business_id'].isin(lista_de_ids)].copy()

In [59]:
df_reviews_res.shape

(1192401, 9)

In [60]:
df_reviews_res.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
22,4zTdZMUfD3wCzpf4U432jA,y5tN0fBBzCAXMY9pmctA5w,R2PSCHpF2zt0lOKeZtyfhg,3.0,0,0,0,The price for this buffet was $27.99. dependin...,2020-02-02 05:42:56


In [61]:
#Dejar la fecha en formato año/mes/día: 
df_reviews_res['date'] = pd.to_datetime(df_reviews_res['date']).dt.date

In [63]:
df_reviews_res.sample(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
631947,d7TdwrIJBxHAU9MQ1Dkeew,OYRdd_IBcWaIT9tUfLv0YQ,Bw0cpnfI-Y9QmjWzqS-oFg,5.0,2,0,0,Wonderful food and service. I've been there ar...,2015-10-11


### Corroborar que todos los restaurantes tienen reviews

In [64]:
df_business_Yelp.shape

(15305, 11)

In [65]:
# Obtén una lista de todos los ID de restaurantes únicos en df_business_Yelp
restaurant_ids = df_business_Yelp['business_id'].unique()

# Verifica si cada ID de restaurante está en la lista de ID de reseñas en df_reviews
are_all_restaurant_ids_in_reviews = all(df_reviews_res['business_id'].isin(restaurant_ids))

if are_all_restaurant_ids_in_reviews:
    print("Todos los ID de restaurantes se encuentran en la lista de reseñas.")
else:
    print("Al menos un ID de restaurante no se encuentra en la lista de reseñas.")

Todos los ID de restaurantes se encuentran en la lista de reseñas.


In [35]:
df_business_Yelp.to_csv('Restaurantes_Yelp.csv',index=False)

In [67]:
df_reviews_res.to_csv('Restaurantes_Yelp_Reviews.csv',index=False)

USER: Data del usuario incluyendo referencias a otros usuarios amigos y a toda la metadata asociada al usuario.

In [68]:
archivo_parquet = "../PROYECTO_FINAL_EQUIPO/Yelp/user-001.parquet"

# Abre el archivo Parquet y crea un DataFrame.

df_user = pd.read_parquet(archivo_parquet)

In [69]:
df_user.shape

(2105597, 22)

In [70]:
#Analizar duplicados:
duplicados = df_user[df_user['user_id'].duplicated()]
print(duplicados.shape)
# Ordena los registros duplicados por 'user_id'
duplicados_ordenados = duplicados.sort_values(by='user_id')

(117700, 22)


In [71]:
duplicados_ordenados.head(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
2017202,---2PmXbF47D870stH1jqA,Susan,36,2012-10-24 13:20:46,63,4,36,,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",3,...,0,0,0,0,0,2,0,0,2,0
2080502,---2PmXbF47D870stH1jqA,Susan,36,2012-10-24 13:20:46,63,4,36,,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",3,...,0,0,0,0,0,2,0,0,2,0


In [72]:
#Eliminar duplicados:
df_user = df_user.drop_duplicates(subset='user_id') 

#### Filtrar usuarios que han reseñado restaurantes

In [73]:
#Hay 1,987,897  de usuarios únicos
df_user.shape

(1987897, 22)

In [74]:
df_user.sample(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
605669,8Xzr-qx7D9vazyAkEB1d-Q,Dawn,13,2009-12-09 04:57:18,14,3,6,,"HrPDIqmVfzztD0z3i31Wcg, jRXr_ynus4PZEofYe_xgMg...",0,...,0,0,0,0,0,0,0,0,0,0
626967,t_unmRFbvisuSuWW_HFHSw,Tuan,156,2011-01-22 01:45:53,191,31,109,"2016,2017,2018,2019,20,20,2021","rWo7TEAHu4wMjNUS3LsXtg, jMsWraiOIPIBmaLSSV8WSQ...",12,...,0,0,0,0,6,14,19,19,0,24


In [77]:
#Ahora filtramos los usuarios únicamente que han echo alguna review
# en  los restaurantes. 
lista_de_id = df_reviews_res['user_id'].tolist()

# Filtrar basándose en la lista de IDs que aparecen en Restaurant 
df_user_review_r = df_user[df_user['user_id'].isin(lista_de_id)].copy()

In [93]:
#Dejar la fecha en formato año/mes/día: 
df_user['yelping_since'] = pd.to_datetime(df_user['yelping_since']).dt.date

In [95]:
#Nos quedamos con 589,377 usuarios. 
df_user_review_r.shape

(589592, 22)

In [96]:
df_user_review_r.to_csv('Restaurantes_User_Yelp.csv',index=False)

### Tips (consejos) escritos por el usuario. 
Los tips son más cortas que las reseñas y tienden a dar sugerencias rápidas.


In [82]:
ruta_archivo_json = "/Users/alejandrobenjaminnunezguzman/Desktop/PROYECTO_FINAL_EQUIPO/Yelp/tip.json"

lista_objetos_json = []

# Abre el archivo JSON y lee línea por línea
with open(ruta_archivo_json, "r") as archivo_json:
    for linea in archivo_json:
        try:
            # Intenta cargar la línea como un objeto JSON
            objeto_json = json.loads(linea)
            lista_objetos_json.append(objeto_json)
        except json.JSONDecodeError as e:
            print(f"Error al cargar línea: {linea}")
            continue

# Crea un DataFrame a partir de la lista de objetos JSON
df_tip = pd.DataFrame(lista_objetos_json)

In [97]:
df_tip.shape

(908915, 5)

In [105]:
df_tip.head(1)

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0


In [98]:
#Ahora filtramos los usuarios que únicamente han dejado tips
#en  los restaurantes. 
df_tip_r = df_tip[df_tip['user_id'].isin(lista_de_id)].copy()

In [102]:
#Dejar la fecha en formato año/mes/día: 
df_tip_r['date'] = pd.to_datetime(df_tip_r['date']).dt.date

In [103]:
#Obtenemos 640,808 tips de diferentes usuarios
df_tip_r.shape

(640847, 5)

In [116]:
df_tip_r.shape

(640847, 5)

In [117]:
df_tip_r.to_csv('Restaurantes_Yelp_Tips.csv',index=False)